<span style="font-family:Ebrima; font-weight:bold;font-size:2.4em;"> AIML MODULE PROJECT - NLP Module Project 1 - Statistical NLP

<span style="font-family:Ebrima; font-weight:bold;font-size:1.5em;color:#3A8D71;"> SUBMITTED BY : RANJITHKUMAR S

<span style="font-family:Ebrima; font-weight:bold;font-size:2.5em;color:#b44b97;"> Part A

<span style="font-family:Ebrima; font-weight:bold;font-size:1em;color:#3A8D71;">DOMAIN:</span> Digital content management

<span style="font-family:Ebrima; font-weight:bold;font-size:1em;color:#3A8D71;">CONTENT:</span>Classification is probably the most popular task that you would deal with in real life. Text in the form of blogs, posts, articles, etc.
are written every second. It is a challenge to predict the information about the writer without knowing about him/her. We are going to create a classifier that predicts multiple features of the author of a given text. We have designed it as a Multi label classification problem.


<span style="font-family:Ebrima; font-weight:bold;font-size:1em;color:#3A8D71;">DATA DESCRIPTION:</span> 
Over 600,000 posts from more than 19 thousand bloggers The Blog Authorship Corpus consists of the collected posts of
19,320 bloggers gathered from blogger.com in August 2004. The corpus incorporates a total of 681,288 posts and over 140 million words - or
approximately 35 posts and 7250 words per person. Each blog is presented as a separate file, the name of which indicates a blogger id# and
the blogger’s self-provided gender, age, industry, and astrological sign. (All are labelled for gender and age but for many, industry and/or sign is
marked as unknown.) All bloggers included in the corpus fall into one of three age groups:
• 8240 "10s" blogs (ages 13-17),
• 8086 "20s" blogs(ages 23-27) and
• 2994 "30s" blogs (ages 33-47)
• For each age group, there is an equal number of male and female bloggers. Each blog in the corpus includes at least 200 occurrences of
common English words. All formatting has been stripped with two exceptions. Individual posts within a single blogger are separated by the
date of the following post and links within a post are denoted by the label url link.

<span style="font-family:Ebrima; font-weight:bold;font-size:1em;color:#3A8D71;">PROJECT OBJECTIVE:</span> To build a NLP classifier which can use input text parameters to determine the label/s of the blog. Specific to this case
study, you can consider the text of the blog: ‘text’ feature as independent variable and ‘topic’ as dependent variable.

In [1]:
#importing necessary python libraries
import os
import re
import nltk
import spacy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

#importing Tensorflow
import tensorflow as tf

#importing langdetect
from langdetect import detect

#import train test split model from sklearn model selection to create train and test dataset
from sklearn.model_selection import train_test_split

#importing the stopwords helps to remove the stopwords from the corpus
from nltk.corpus import stopwords

# Helps to pad the sequences into the same length
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Layers that are used to implement the LSTM model
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

#importing label encode to convert the labels in the target columns into a number 
from sklearn.preprocessing import LabelEncoder

#importing countvectorizer from sklearn feature extracting for converting text data into numberic
from sklearn.feature_extraction.text import CountVectorizer

#importing the MulitnomialNB function from sklearn naive bayes for the classification
#importing the performance metrics from sklearn metrics 
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,precision_score,recall_score,roc_auc_score

#import TF-IDF Vectorizer from sklearn feature extraction
from sklearn.feature_extraction.text import TfidfVectorizer

#importing gensim for vector embeddings and importing 
import gensim
from gensim.models import Word2Vec

#importing minmax scaler from sklearn preprocessing for normalize the features
from sklearn.preprocessing import MinMaxScaler

#importing the base models from the sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

<span style="font-family:Ebrima; font-weight:bold;font-size:1.5em;color:#b44b97;">1 A - Clearly write outcome of data analysis

In [2]:
# reading blogtext csv unsinf pandas

blogtxt = pd.read_csv('blogtext.csv')

In [3]:
blogtxt.shape

(681284, 7)

In [4]:
#making copy of the dataset to workon
data = blogtxt.sample(30000)

In [5]:
#EDA
#size of dataset
shape = data.shape
print("\033[1m1. Shape Report:\033[0m \n\tThe Dataset as {} Rows and {} Columns\n".format(shape[0],shape[1]))

#Datas
print("\033[0\1m2. Data's in the Dataset\033[0m\n")
#Presenting the data
print("\033[1m2.1 First 5 Rows from the dataset:\033[0m")
display(data.head(5))
print("\033[1m2.2 Last 5 Rows from the dataset\033[0m")
display(data.tail(5))

#Info
print("\033[1m3. Infomation about data in the dataset:\033[0m")
display(data.info())

#describe
print('\033[1m4. Distribution of the Numberic Columns:\033[0m')
display(data.describe())

#duplicate
print('\033[01m5. Number of Duplicate Entries in the dataset:\033[0m {}'.format(data.duplicated().sum()))


1. Shape Report: 
	The Dataset as 30000 Rows and 7 Columns

[0m2. Data's in the Dataset

2.1 First 5 Rows from the dataset:


,id,gender,age,topic,sign,date,text
320259,947299,male,24,Law,Gemini,"24,August,2003","WWE Summer Slam was phenominal, here's my r..."
111750,1662633,female,23,Engineering,Pisces,"22,June,2004",we must have been posting at the same t...
180457,1093457,male,16,Technology,Virgo,"22,July,2003","I'm bored, Ben is cool"
458390,518116,female,27,indUnk,Pisces,"29,March,2004",I'm so glad that worked! Anyway...so s...
581214,3387580,female,33,indUnk,Virgo,"18,May,2004",I figured it out this time but I ...


2.2 Last 5 Rows from the dataset


,id,gender,age,topic,sign,date,text
367097,1119280,female,27,indUnk,Gemini,"01,August,2004",This afternoon I got a phone call from ...
551038,1125546,male,27,Law,Aries,"20,July,2004",Though a lot of lawyers are themselves ...
244020,1506385,female,27,Internet,Gemini,"11,May,2004",Haven't seen a bollywood flick in 2...
438062,1853281,male,23,Student,Cancer,"02,October,2003","I know, I know bad poetry, i promised i..."
139272,3552447,male,15,Student,Aquarius,"14,July,2004",umm...i forgot what i was gonna put her...


3. Infomation about data in the dataset:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 320259 to 139272
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      30000 non-null  int64 
 1   gender  30000 non-null  object
 2   age     30000 non-null  int64 
 3   topic   30000 non-null  object
 4   sign    30000 non-null  object
 5   date    30000 non-null  object
 6   text    30000 non-null  object
dtypes: int64(2), object(5)
memory usage: 1.8+ MB


None

4. Distribution of the Numberic Columns:


,id,age
count,3.000000e+04,30000.00000
mean,2.401867e+06,23.99980
std,1.244008e+06,7.85341
min,5.114000e+03,13.00000
25%,1.246850e+06,17.00000
50%,2.615621e+06,24.00000
75%,3.525448e+06,27.00000
max,4.334761e+06,48.00000


5. Number of Duplicate Entries in the dataset: 33


<span style="font-family:Ebrima; font-weight:bold;font-size:1em;color:#3845C7;"> Observations: </span> 

1. The Dataset blog text as 681284 Rows and 7 Columns reducing the data by sample 20000 rows
2. Columns Details:

    2.1 In the 7 columns, 2 int64 datatype id (unique number of blog) & age (age of the person making the blog)
    
    2.2 Remaining are the object data type columns gender(gender of a person), topic(the blog based on), sign(posted name), date (date posted), text(context onf the blog)
3. for information function that there is not Null values in the dataset and columns
4. 4686 duplicate entiries in the dataset

In [6]:
#finding unique entire in the traget columns
unique = data['topic'].unique()
print(unique)

['Law' 'Engineering' 'Technology' 'indUnk' 'LawEnforcement-Security'
 'Fashion' 'Communications-Media' 'Student' 'Internet' 'Arts' 'Government'
 'Publishing' 'Non-Profit' 'Education' 'RealEstate' 'Banking' 'Marketing'
 'Tourism' 'Science' 'HumanResources' 'Religion' 'BusinessServices'
 'Consulting' 'Chemicals' 'Agriculture' 'Automotive' 'Accounting'
 'Advertising' 'Sports-Recreation' 'Military' 'Construction' 'Biotech'
 'Architecture' 'InvestmentBanking' 'Telecommunications' 'Environment'
 'Transportation' 'Manufacturing' 'Museums-Libraries' 'Maritime']


In [7]:
len(unique)

40

<span style="font-family:Ebrima; font-weight:bold;font-size:1em;color:#3845C7;"> Observations: </span> The Target Columns topic as 40 different topics

<span style="font-family:Ebrima; font-weight:bold;font-size:1.5em;color:#b44b97;">1 B - Clean the Structured Data

In [8]:
data.isna().sum()

id        0
gender    0
age       0
topic     0
sign      0
date      0
text      0
dtype: int64

<span style="font-family:Ebrima; font-weight:bold;font-size:1em;color:#3845C7;"> Observations: </span> No Null Values is observered from the isna function

In [9]:
#defining function to detect the lang of the text column
#passing data
def detect_english(data):
    try:
        return detect(data) == 'en' #detecting lang is en
    except:
        return False #else return False

In [10]:
data['lang'] = data['text'].apply(detect_english) #adding lang column in the data

In [11]:
noneng = data.loc[data['lang'] == False].index #making the list of indexes of non english
print('Non English Text:',len(noneng)) #printing the length of the list

Non English Text: 1351


In [12]:
data = data.drop(noneng) #droping the columns

<span style="font-family:Ebrima; font-weight:bold;font-size:1.5em;color:#b44b97;">2 A - Eliminate All special Characters and Numbers

In [13]:
#function for removing special character and number 
def remove_sn(data):
    return re.sub(r"[^a-zA-Z]", " ",data)

data['Clean Text'] = data['text'].apply(remove_sn) #applying the function to the text columns

In [14]:
data.head()

,id,gender,age,topic,sign,date,text,lang,Clean Text
320259,947299,male,24,Law,Gemini,"24,August,2003","WWE Summer Slam was phenominal, here's my r...",True,WWE Summer Slam was phenominal here s my r...
111750,1662633,female,23,Engineering,Pisces,"22,June,2004",we must have been posting at the same t...,True,we must have been posting at the same t...
180457,1093457,male,16,Technology,Virgo,"22,July,2003","I'm bored, Ben is cool",True,I m bored Ben is cool
458390,518116,female,27,indUnk,Pisces,"29,March,2004",I'm so glad that worked! Anyway...so s...,True,I m so glad that worked Anyway so s...
581214,3387580,female,33,indUnk,Virgo,"18,May,2004",I figured it out this time but I ...,True,I figured it out this time but I ...


<span style="font-family:Ebrima; font-weight:bold;font-size:1.5em;color:#b44b97;">2 B - Lowercase all textual data

In [15]:
#function for changing the text into lower case
def remove_lc(data):
    return data.lower()

data['Clean Text'] = data['Clean Text'].apply(remove_lc) #applying the function to the clear text columns

In [16]:
data.head()

,id,gender,age,topic,sign,date,text,lang,Clean Text
320259,947299,male,24,Law,Gemini,"24,August,2003","WWE Summer Slam was phenominal, here's my r...",True,wwe summer slam was phenominal here s my r...
111750,1662633,female,23,Engineering,Pisces,"22,June,2004",we must have been posting at the same t...,True,we must have been posting at the same t...
180457,1093457,male,16,Technology,Virgo,"22,July,2003","I'm bored, Ben is cool",True,i m bored ben is cool
458390,518116,female,27,indUnk,Pisces,"29,March,2004",I'm so glad that worked! Anyway...so s...,True,i m so glad that worked anyway so s...
581214,3387580,female,33,indUnk,Virgo,"18,May,2004",I figured it out this time but I ...,True,i figured it out this time but i ...


<span style="font-family:Ebrima; font-weight:bold;font-size:1.5em;color:#b44b97;">2 C - Remove all Stopwords

In [17]:
%%time
#function for removing the stopwords
def remove_sw(data):
    return ' '.join([word for word in data.split() if word not in stopwords.words('english')])

data['Clean Text'] = data['Clean Text'].apply(remove_sw) #applying the function to the clear Text field

CPU times: total: 18min 21s
Wall time: 22min 44s


In [18]:
data.head()

,id,gender,age,topic,sign,date,text,lang,Clean Text
320259,947299,male,24,Law,Gemini,"24,August,2003","WWE Summer Slam was phenominal, here's my r...",True,wwe summer slam phenominal review first match ...
111750,1662633,female,23,Engineering,Pisces,"22,June,2004",we must have been posting at the same t...,True,must posting time cause almost lost entire blo...
180457,1093457,male,16,Technology,Virgo,"22,July,2003","I'm bored, Ben is cool",True,bored ben cool
458390,518116,female,27,indUnk,Pisces,"29,March,2004",I'm so glad that worked! Anyway...so s...,True,glad worked anyway sis make britney spears con...
581214,3387580,female,33,indUnk,Virgo,"18,May,2004",I figured it out this time but I ...,True,figured time figure find place put post illite...


<span style="font-family:Ebrima; font-weight:bold;font-size:1.5em;color:#b44b97;">2 D - Remove all extra white spaces

In [19]:
data['Clean Text'] = data['Clean Text'].str.strip() #removing extra white spaces from the text using str strip function

In [20]:
data.head()

,id,gender,age,topic,sign,date,text,lang,Clean Text
320259,947299,male,24,Law,Gemini,"24,August,2003","WWE Summer Slam was phenominal, here's my r...",True,wwe summer slam phenominal review first match ...
111750,1662633,female,23,Engineering,Pisces,"22,June,2004",we must have been posting at the same t...,True,must posting time cause almost lost entire blo...
180457,1093457,male,16,Technology,Virgo,"22,July,2003","I'm bored, Ben is cool",True,bored ben cool
458390,518116,female,27,indUnk,Pisces,"29,March,2004",I'm so glad that worked! Anyway...so s...,True,glad worked anyway sis make britney spears con...
581214,3387580,female,33,indUnk,Virgo,"18,May,2004",I figured it out this time but I ...,True,figured time figure find place put post illite...


<span style="font-family:Ebrima; font-weight:bold;font-size:1.5em;color:#b44b97;">3 A - Create dependent and independent variables

In [21]:
#making the text as depenent variable & the Topic of the text into independent variable 
X = data['Clean Text']
y = data['topic']

In [22]:
le = LabelEncoder() #creating instance of the label encoder function
le.fit(y) #fit the target data into the labelencoder instance
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_))) #making label name mapping dictionary by zipping the class and transformer class values from label encoder 

In [23]:
y = y.map(le_name_mapping) #mapping y value as y

<span style="font-family:Ebrima; font-weight:bold;font-size:1.5em;color:#b44b97;">3 B - Split data into train and test.

In [24]:
Xtrain,Xtest,ytrain,ytest = train_test_split(X,y,test_size=0.3,random_state=12) #creating the training and testing dataset using train_test_split function from X & y

In [25]:
#printing the shapes of all the training and testing dataset
display(Xtrain.shape)
display(Xtest.shape)
display(ytrain.shape)
display(ytest.shape)

(20054,)

(8595,)

(20054,)

(8595,)

<span style="font-family:Ebrima; font-weight:bold;font-size:1.5em;color:#b44b97;">3 C - Vectorize data using any one vectorizer.

In [26]:
#creating instances for the counter vectorizer function with parameters Max feature and ngram range
cv = CountVectorizer(max_features=5000,ngram_range=(1,4))
X_train = cv.fit_transform(Xtrain).toarray() #transforming the Xtrain using count vectorizer function and making it as array
X_test = cv.fit_transform(Xtest).toarray() #transforming the Xtest using count vectorizer function and making it as array

In [27]:
cv.vocabulary_ #displaying the vocabulary from the count vectorizer model

{'well': 4805,
 'back': 293,
 'weeks': 4800,
 'cell': 630,
 'phone': 3251,
 'anyone': 171,
 'much': 2869,
 'give': 1756,
 'ring': 3681,
 'ta': 4289,
 'cell phone': 631,
 'yeah': 4966,
 'last': 2398,
 'couple': 917,
 'days': 1019,
 'went': 4820,
 'see': 3828,
 'matt': 2710,
 'sharp': 3897,
 'play': 3298,
 'sunday': 4248,
 'fun': 1679,
 'except': 1394,
 'fact': 1439,
 'girlfriend': 1754,
 'know': 2344,
 'honestly': 2043,
 'happy': 1942,
 'whole': 4841,
 'hell': 1986,
 'lot': 2606,
 'better': 391,
 'seeing': 3834,
 'must': 2884,
 'say': 3775,
 'nicely': 2981,
 'got': 1832,
 'girl': 1753,
 'keeping': 2301,
 'hair': 1920,
 'cut': 973,
 'wearing': 4784,
 'nice': 2980,
 'clothing': 755,
 'shape': 3893,
 'every': 1369,
 'morning': 2847,
 'complete': 820,
 'long': 2578,
 'clock': 746,
 'ripped': 3685,
 'dirty': 1129,
 'clothes': 754,
 'guy': 1905,
 'saw': 3774,
 'year': 4967,
 'ago': 76,
 'jealous': 2239,
 'make': 2655,
 'eyes': 1430,
 'smile': 4011,
 'really': 3550,
 'wish': 4873,
 'could': 89

<span style="font-family:Ebrima; font-weight:bold;font-size:1.5em;color:#b44b97;">3 D - Build a base model for Supervised Learning - Classification.

In [28]:
#creating the instance of MultinomialNB model with alpha parameter 
model=MultinomialNB(alpha=.01)
model.fit(X_train,ytrain) # fit the training data in the model instance
ypred = model.predict(X_test) #making prediction from the model

<span style="font-family:Ebrima; font-weight:bold;font-size:1.5em;color:#b44b97;">3 E - Clearly print Performance Metrics.

In [29]:
#Performance metrics
score = accuracy_score(ytest, ypred) #calculating accuracy score 
precision = precision_score(ytest, ypred,average='weighted') #calculating precision score
recall = recall_score(ytest,ypred,average='weighted') #calculating recall score
#printing the performance metrics
print("Accuracy Score:   %0.3f" % score)
print("Precision Score:   %0.3f" % precision)
print("Recall Score:   %0.3f" % recall)

Accuracy Score:   0.297
Precision Score:   0.216
Recall Score:   0.297


<span style="font-family:Ebrima; font-weight:bold;font-size:1.5em;color:#b44b97;">4 A - Experiment with other vectorisers.

In [30]:
#creating instance for TF-IDF vectorizer with parameters
tf_idf_vectorizer = TfidfVectorizer(max_features=5000,ngram_range=(1,4))
X_tf_idf = tf_idf_vectorizer.fit_transform(X).toarray() #converting the text data X into numeric using the TF-IDF and making into array

In [31]:
Xtrain_tf_idf,Xtest_tf_idf,ytrain,ytest = train_test_split(X_tf_idf,y,test_size=0.3,random_state=12) #creating the training and testing dataset using train_test_split function from X_tf_idf & y 

In [32]:
model.fit(Xtrain_tf_idf,ytrain) # fit the training data in the base model 
ypred = model.predict(Xtest_tf_idf) #making prediction from the model

In [33]:
#Performance metrics
score = accuracy_score(ytest, ypred) #calculating accuracy score 
precision = precision_score(ytest, ypred,average='weighted') #calculating precision score
recall = recall_score(ytest,ypred,average='weighted')#calculating recall score
#printing the performance metrics
print("Accuracy Score:   %0.3f" % score)
print("Precision Score:   %0.3f" % precision)
print("Recall Score:   %0.3f" % recall)

Accuracy Score:   0.388
Precision Score:   0.366
Recall Score:   0.388


In [34]:
# Applying Word2 vec Wordembedding
#perparing data for word Embedding
words_list = []
for i in data['Clean Text']:
    li = list(i.split(" "))
    words_list.append(li)

In [35]:
# Model creation   
model= Word2Vec(words_list, min_count = 1, workers = 4)
# saving the model
model.save("word2vec.model")

In [36]:
words = model.wv.index_to_key #getting the values from the model index to key function to words variable
print(len(words)) #printing length of the words

104838


In [37]:
wvsd = model.wv[words] #getting model  words indexes

In [38]:
#function for creating word embedding
#defining function with word,model,vocabulary and num of features
def average_word_vectors(words, model, vocabulary, num_features):
    feature_vector = np.zeros((num_features,), dtype="float64") #generting np zero array with the length of the number of features
    nwords = 0. #nwords variable with 0
    #for loop from the words
    for word in words:
        if word in vocabulary: #checking for word in the vocabulary
            nwords = nwords + 1. #add nword into 1
            feature_vector = np.add(feature_vector, model.wv[word]) #add feature vector value and word index into the feature vector zero array 
    
    if nwords: #checking for nwords values
        feature_vector = np.divide(feature_vector, nwords) #dividing the feature vector 
        
    return feature_vector # returing the feature vector

def averaged_word_vectorizer(corpus, model, num_features): #call the function
    
    vocabulary = set(model.wv.index_to_key) #making a set name vocabulary from words from the model
    
    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features) for tokenized_sentence in corpus] #making a list from corpus token sentences from the average word vector function
    
    return np.array(features) #returing the features as np array 

In [39]:
feature_size = 100 #defining the feature size

# get document level embeddings
# averaged_word_vectorizer and making w2v_feature_array
w2v_feature_array = averaged_word_vectorizer(corpus = words_list, model = model,num_features = feature_size)
pd.DataFrame(w2v_feature_array)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.273184,0.098726,0.280400,0.183860,-0.208212,-0.760461,0.449373,0.585213,-0.156152,-0.579903,...,0.596796,0.059671,-0.354892,0.135558,0.376102,0.074805,0.159273,-0.316944,-0.243655,-0.050085
1,-0.285943,0.072359,0.450923,-0.095572,-0.305558,-1.026383,0.642953,0.570524,-0.263432,-1.126519,...,0.704894,0.132940,-0.695402,0.452642,0.429551,0.310148,-0.114320,-0.480260,-0.271993,-0.240440
2,-0.063803,0.336951,-0.130620,0.613464,0.094118,-1.157574,0.204765,0.627546,-0.769671,-0.762810,...,2.378349,0.497056,-0.976431,0.352491,-0.982168,-0.154208,-0.279138,-0.478174,-0.170591,-0.066500
3,-0.327360,0.412342,0.492439,0.459046,-0.349096,-1.045358,0.598068,0.602835,-0.154824,-0.886743,...,0.895395,0.026481,-0.680626,0.051992,0.219147,0.589273,-0.101161,-0.078259,-0.195583,0.036364
4,-0.418884,0.190261,0.516711,0.060653,-1.040180,-0.887904,0.852759,0.507574,-0.209439,-0.979611,...,0.875724,0.082292,-0.547058,-0.121155,0.777290,1.051435,0.211437,-0.307468,-0.102203,-0.484032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28644,-1.279000,0.042484,-0.324437,0.379307,0.471290,-1.083894,0.023564,0.795872,0.287041,-0.927514,...,1.528844,0.208263,-0.148910,0.012280,0.222036,1.162293,-0.110773,-0.518668,-0.849841,-0.067744
28645,-0.480383,0.394851,0.609007,-0.063965,0.339788,-0.383633,0.123862,0.777806,-0.090629,-0.628620,...,0.356375,-0.061860,-0.549273,-0.033639,0.818167,0.504199,0.130275,-0.399541,0.327082,-0.354515
28646,-0.318632,-0.158180,0.196987,-0.039179,-0.195626,-0.664604,0.556909,0.367798,-0.354340,-0.625903,...,0.779261,-0.003109,-0.468815,0.199878,0.200744,0.016839,0.059738,-0.587869,-0.062424,-0.116664
28647,-0.180786,0.149658,0.468319,0.182842,-0.987062,-1.087828,0.879041,0.546562,-0.237751,-1.116019,...,0.859150,-0.253775,-0.929214,0.167125,0.221436,0.221296,-0.096573,-0.586891,-0.095332,-0.023362


In [40]:
Xtrain_w2v,Xtest_w2v,ytrain,ytest = train_test_split(w2v_feature_array,y, test_size=0.3, random_state=12) #creating the training and testing dataset using train_test_split function from w2v_feature_array & y 

In [41]:
#creating a instance for MinMaxScaler 
scaler = MinMaxScaler()
Xtrain_w2v = scaler.fit_transform(Xtrain_w2v) #fitand Transforming the Xtrain data
Xtest_w2v = scaler.fit_transform(Xtest_w2v) #fir and Transforming the Xtest Data

In [42]:
#creating instance for the MultinomialNB base mode
model = MultinomialNB()
model.fit(Xtrain_w2v,ytrain) #fit the training data into model
ypred = model.predict(Xtest_w2v) #making prediction from the model

In [43]:
#Performance metrics
score = accuracy_score(ytest, ypred) #calculating accuracy score 
precision = precision_score(ytest, ypred,average='weighted') #calculating precision score
recall = recall_score(ytest,ypred,average='weighted') #calculating recall score
#printing the performance metrics
print("Accuracy Score:   %0.3f" % score)
print("Precision Score:   %0.3f" % precision)
print("Recall Score:   %0.3f" % recall)

Accuracy Score:   0.367
Precision Score:   0.134
Recall Score:   0.367


<span style="font-family:Ebrima; font-weight:bold;font-size:1.5em;color:#b44b97;">4 B - Build classifier Models using other algorithms than base model.

In [44]:
#making pipeline by creating list with model name and instances
model_pipeline = []
model_pipeline.append(['Logistic Regression',LogisticRegression()]) #appending the logistic regression
model_pipeline.append(['KNN',KNeighborsClassifier()])  #appending the KNeighborsClassifier
model_pipeline.append(['Decision Tree Classifier',DecisionTreeClassifier()])  #appending the Decision Tree Classifier
model_pipeline.append(['Gaussian',GaussianNB()]) # #appending the GaussianNB

Buliding Classifier Model for Count Vectorizer method

In [45]:
#model Evaluation
model_eval = [] #empty list to store the meterics for the pipeline

for model in model_pipeline: #for loop thru the model list
    value = [] #empty list
    model[1].fit(X_train,ytrain) #fiting traing data into the selected model
    ypred = model[1].predict(X_test) # model preodction
    #perfromace metrics
    acc_score = accuracy_score(ypred,ytest)
    pre_score = precision_score(ytest, ypred,average='weighted')
    rel_score = recall_score(ytest,ypred,average='weighted')
    value.append(model[0])
    value.append(acc_score)
    value.append(pre_score)
    value.append(rel_score)
    model_eval.append(value)
result_with_count = pd.DataFrame(model_eval,columns=['Model','Accuracy Score','Precision Score','Recall Score'])

Buliding Classifier Model for TF-IDF Vectorizer method

In [46]:
%%time
#model Evaluation
model_eval = [] #empty list to store the meterics for the pipeline

for model in model_pipeline: #for loop thru the model list
    value = [] #empty list
    model[1].fit(Xtrain_tf_idf,ytrain) #fiting traing data into the selected model
    ypred = model[1].predict(Xtest_tf_idf) # model preodction
    #perfromace metrics
    acc_score = accuracy_score(ypred,ytest)
    pre_score = precision_score(ytest, ypred,average='weighted')
    rel_score = recall_score(ytest,ypred,average='weighted')
    value.append(model[0])
    value.append(acc_score)
    value.append(pre_score)
    value.append(rel_score)
    model_eval.append(value)
result_with_tfidf = pd.DataFrame(model_eval,columns=['Model','Accuracy Score','Precision Score','Recall Score'])

CPU times: total: 4min 46s
Wall time: 3min 21s


Buliding Classifier Model for Word2vec method

In [47]:
%%time
#model Evaluation
model_eval = [] #empty list to store the meterics for the pipeline

for model in model_pipeline:
    value = [] #empty list
    model[1].fit(Xtrain_w2v,ytrain) #fiting traing data into the selected model
    ypred = model[1].predict(Xtest_w2v) # model preodction
    #perfromace metrics
    acc_score = accuracy_score(ypred,ytest)
    pre_score = precision_score(ytest, ypred,average='weighted')
    rel_score = recall_score(ytest,ypred,average='weighted')
    value.append(model[0])
    value.append(acc_score)
    value.append(pre_score)
    value.append(rel_score)
    model_eval.append(value)
result_with_wv = pd.DataFrame(model_eval,columns=['Model','Accuracy Score','Precision Score','Recall Score'])

CPU times: total: 11.8 s
Wall time: 7.74 s


<span style="font-family:Ebrima; font-weight:bold;font-size:1.5em;color:#b44b97;">4 D - Clearly print Performance Metrics.

In [48]:
result_with_count #displaying the performace of the various models on count vectorizer method

,Model,Accuracy Score,Precision Score,Recall Score
0,Logistic Regression,0.232344,0.202090,0.232344
1,KNN,0.252356,0.198510,0.252356
2,Decision Tree Classifier,0.223153,0.205148,0.223153
3,Gaussian,0.209308,0.201089,0.209308


In [49]:
result_with_tfidf #displaying the performace of the various models on TF-IDF

,Model,Accuracy Score,Precision Score,Recall Score
0,Logistic Regression,0.381035,0.275392,0.381035
1,KNN,0.363467,0.252387,0.363467
2,Decision Tree Classifier,0.251774,0.225601,0.251774
3,Gaussian,0.131937,0.241478,0.131937


In [50]:
result_with_wv ##displaying the performace of the various models on Word2vec

,Model,Accuracy Score,Precision Score,Recall Score
0,Logistic Regression,0.368121,0.241623,0.368121
1,KNN,0.260733,0.224996,0.260733
2,Decision Tree Classifier,0.178709,0.201388,0.178709
3,Gaussian,0.020361,0.217520,0.020361


<span style="font-family:Ebrima; font-weight:bold;font-size:1.5em;color:#b44b97;">5 A - Which vectorizer performed better? Probable reason?.

## TF-IDF(term frequency-inverse document frequency)

### From the above evaluation the the TF-IDF(term frequency-inverse document frequency) vectorization give higher accuracy score compare to all the other vectorization methods

<span style="font-family:Ebrima; font-weight:bold;font-size:1.5em;color:#b44b97;">5 B - Which model outperformed? Probable reason?

## Logistic Regression

### From the above model comparision the Logistic Refression give higher accuracy score with the TF-IDF data vectorization method gives 38.5 accuray score

<span style="font-family:Ebrima; font-weight:bold;font-size:1.5em;color:#b44b97;">5 C - Which parameter/hyperparameter significantly helped to improve performance? Probable reason?.

### Vectorization method play major role affecting the perfomance of the model, using different vectorization methods with base model give base average

<span style="font-family:Ebrima; font-weight:bold;font-size:1.5em;color:#b44b97;">5 D - According to you, which performance metric should be given most importance, why?.

### ROC AUC 
ROC curves are widely used to compare and evaluate different classification algorithms.

ROC curve is widely used when the dataset is imbalanced.

ROC curves are also used in verification of forecasts in meteorology

<span style="font-family:Ebrima; font-weight:bold;font-size:2.5em;color:#b44b97;"> Part B

<span style="font-family:Ebrima; font-weight:bold;font-size:1em;color:#3A8D71;">DOMAIN:</span> Customer support

<span style="font-family:Ebrima; font-weight:bold;font-size:1em;color:#3A8D71;">CONTENT:</span> Great Learning has a an academic support department which receives numerous support requests every day throughout the year.
Teams are spread across geographies and try to provide support round the year. Sometimes there are circumstances where due to heavy
workload certain request resolutions are delayed, impacting company’s business. Some of the requests are very generic where a proper
resolution procedure delivered to the user can solve the problem. Company is looking forward to design an automation which can interact with
the user, understand the problem and display the resolution procedure [ if found as a generic request ] or redirect the request to an actual human
support executive if the request is complex or not in it’s database.

<span style="font-family:Ebrima; font-weight:bold;font-size:1em;color:#3A8D71;">DATA DESCRIPTION:</span> 
A sample corpus is attached for your reference. Please enhance/add more data to the corpus using your linguistics skills.

<span style="font-family:Ebrima; font-weight:bold;font-size:1em;color:#3A8D71;">PROJECT OBJECTIVE:</span> Design a python based interactive semi - rule based chatbot which can do the following:

<span style="font-family:Ebrima; font-weight:bold;font-size:1.5em;color:#b44b97;">1 - Start chat session with greetings and ask what the user is looking for.    

<span style="font-family:Ebrima; font-weight:bold;font-size:1.5em;color:#b44b97;">2 - Accept dynamic text based questions from the user. Reply back with relevant answer from the designed corpus.    

<span style="font-family:Ebrima; font-weight:bold;font-size:1.5em;color:#b44b97;">3 - End the chat session only if the user requests to end else ask what the user is looking for. Loop continues till the user asks to end it.    

# Chatbot Buliding

1. Running a Chatbot "greet the user and ask what the user looking for"
2. Dynamic textbased
    For Dynamic textproecssing need to build the model that can process the complex text
    1. Text Case - Handling upper and lower case
    
    ## Text preprocessing
    
    2. tokenization
    
    3. stemming
    
    4. bag of words (BoW)
    
    5. Text Classification model buliding

In [51]:
#importing necessary python libraries
import nltk
import json
import random
import numpy as np
from nltk.stem.porter import *
from nltk.chat.util import Chat,reflections

#importing pickle for pickling the models
import pickle

#import load model 
from keras.models import load_model

from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation

In [52]:
#creating pairs and reflections for the nltk chat function
pairs = [
[
    r"hello",
    ["Hello %1, How are you today ?",]
],
[
    r"my name is (.*)",
    ["Hello %1, How are you today ?",]
],
[
    r"what is your name ?",
    ["My name is Chatbot and I'm a chatbot ?",]
],
[
    r"how are you ?",
    ["I'm doing good\nHow about You ?",]
],
[
    r"what (.*) want ?",
    ["Make me an offer I can't refuse",]       
],

]

reflections= {
 "go"     : "gone",
 "hi"    : "hey there"
}

In [53]:
#defining function name chat bot with introduction and get user input and response  
def chatbot():
    #chatbot intro
    print("Hi, I'm Virual Assistance a chatbot;Please type lowercase English language to start a conversation. Type quit to leave ") #default message at the start
    #chat function with probem
    chat = Chat(pairs, reflections) 
    chat.converse()
if __name__ == "__main__":
    chatbot() #callig the chat function

Hi, I'm Virual Assistance a chatbot;Please type lowercase English language to start a conversation. Type quit to leave 
>my Name is Ranjith
Hello ranjith, How are you today ?
>quit
None


In [54]:
#loading Json file as corpus
data_file=open('GL Bot.json').read() #reading the GL Bot json file as datafile 
intents=json.loads(data_file) #loading the  into intents 

In [55]:
#printing the intents
print(intents)

{'intents': [{'tag': 'Intro', 'patterns': ['hi', 'how are you', 'is anyone there', 'hello', 'whats up', 'hey', 'yo', 'listen', 'please help me', 'i am learner from', 'i belong to', 'aiml batch', 'aifl batch', 'i am from', 'my pm is', 'blended', 'online', 'i am from', 'hey ya', 'talking to you for first time'], 'responses': ['Hello! how can i help you ?'], 'context_set': ''}, {'tag': 'Exit', 'patterns': ['thank you', 'thanks', 'cya', 'see you', 'later', 'see you later', 'goodbye', 'i am leaving', 'have a Good day', 'you helped me', 'thanks a lot', 'thanks a ton', 'you are the best', 'great help', 'too good', 'you are a good learning buddy'], 'responses': ['I hope I was able to assist you, Good Bye'], 'context_set': ''}, {'tag': 'Olympus', 'patterns': ['olympus', 'explain me how olympus works', 'I am not able to understand olympus', 'olympus window not working', 'no access to olympus', 'unable to see link in olympus', 'no link visible on olympus', 'whom to contact for olympus', 'lot of p

In [56]:
# Tokenization 
# empty list to store the values
words=[]
classes=[]
documents=[]
#ingore context list
ignore=['?','!',',',"'s"]

#for loop in intents-intents
for intent in intents['intents']:
  #for loop in intent- patterns
    for pattern in intent['patterns']:
        #tokenize the pattern 
        w=nltk.word_tokenize(pattern)
        words.extend(w) #extenb the words list
        documents.append((w,intent['tag'])) #appending the W and intent tag 
        
        if intent['tag'] not in classes: #if intente tag is not in the classes list
            classes.append(intent['tag']) #add to the classes list

In [57]:
#applying lemmatizer on the words list if word not in ignore list
words=[lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore]
words=sorted(list(set(words))) #making sorted list with set of words
classes=sorted(list(set(classes))) #making sorted list with set of classes
pickle.dump(words,open('words.pkl','wb')) #dumping the words list as pickle file
pickle.dump(classes,open('classes.pkl','wb')) #dumping the classes list as pickle file

In [58]:
#generating training and testing data 

#function bag of words to generate the training and testing data
def bag_of_words():
    training=[] #empty list
    output_empty=[0]*len(classes) #Zero list size of the classes file

    for doc in documents: #creating for loop in the document
        bag=[] #empty list
        pattern=doc[0] #patten equal to the document 0th instance value
        pattern=[ lemmatizer.lemmatize(word.lower()) for word in pattern ] #applying lemmatixation in the pattern list

        for word in words: #for loop in words 
            if word in pattern: #if word in the pattern
                bag.append(1) #making bag list entry into 1
            else:
                bag.append(0) #making bag list entry into 0
        output_row=list(output_empty) #making output row by listing the output_empty list
        output_row[classes.index(doc[1])]=1 #setting index to 1
        training.append([bag,output_row]) #appeding the values into the traing data list

    random.shuffle(training) #shuffle the traing list
    training=np.array(training) #making into array 
    X_train=list(training[:,0]) # making the Xtrain list from the train array 0th entry
    y_train=list(training[:,1]) # making the ytrain list from the train array 1th entry
    
    return X_train,y_train

In [59]:
X_train,y_train = bag_of_words()

In [60]:
#Buliding Model
model=Sequential() #sequential model
model.add(Dense(128,activation='relu',input_shape=(len(X_train[0]),))) #dense layer / input layer with length of the Xtrain rows
model.add(Dropout(0.5)) #dropout layer
model.add(Dense(64,activation='relu')) #hidden layer 1
model.add(Dense(64,activation='relu')) #hidden Layer 2
model.add(Dropout(0.5)) #dropout layer 
model.add(Dense(len(y_train[0]),activation='softmax')) #output layer with softmax activation function

adam=tf.keras.optimizers.Adam(0.001) #defining the adam optimizer function with learing ratw
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy']) #compiling the model
weights=model.fit(np.array(X_train),np.array(y_train),epochs=200,batch_size=10,verbose=1)    #fit the model with X train and y train with 200 epochs
model.save('mymodel.h5',weights) #saving the model

Epoch 1/200
13/13 [==============================] - 1s 4ms/step - loss: 2.1888 - accuracy: 0.1163 
Epoch 2/200
13/13 [==============================] - 0s 4ms/step - loss: 2.1693 - accuracy: 0.1628
Epoch 3/200
13/13 [==============================] - 0s 3ms/step - loss: 2.1147 - accuracy: 0.2171
Epoch 4/200
13/13 [==============================] - 0s 3ms/step - loss: 2.0400 - accuracy: 0.2791
Epoch 5/200
13/13 [==============================] - 0s 3ms/step - loss: 2.0209 - accuracy: 0.2403
Epoch 6/200
13/13 [==============================] - 0s 2ms/step - loss: 1.9592 - accuracy: 0.2791
Epoch 7/200
13/13 [==============================] - 0s 1ms/step - loss: 1.9004 - accuracy: 0.3256
Epoch 8/200
13/13 [==============================] - 0s 2ms/step - loss: 1.8388 - accuracy: 0.3178
Epoch 9/200
13/13 [==============================] - 0s 1ms/step - loss: 1.7892 - accuracy: 0.3256
Epoch 10/200
13/13 [==============================] - 0s 2ms/step - loss: 1.7599 - accuracy: 0.3643
Epoch 11

In [61]:
#loading model,json,words pickle and classes pickle
model = load_model('mymodel.h5')
intents = json.loads(open('GL Bot.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [62]:
#function for cleaning up the words from the user 
def clean_up(sentence):
    sentence_words=nltk.word_tokenize(sentence) #nltk tokenizing the sentences
    sentence_words=[ lemmatizer.lemmatize(word.lower()) for word in sentence_words] #apply lemmatization on the words
    return sentence_words #returning the clean words as sentence words

def create_bow(sentence,words): #creating bag of words function
    sentence_words=clean_up(sentence) #calling the clean function with sentence and assign into the sentence_words
    bag=list(np.zeros(len(words))) #making the list of zero array with size of the words for stroing the BOW
    
    for s in sentence_words: #for loop thru the sentence_words
        for i,w in enumerate(words): #getting index and word
            if w == s:  # word equal to sentence
                bag[i] = 1 #make bag list indexed instance into 1
    return np.array(bag) #return the bag of words list as array

In [63]:
#prediction function
def predict_class(sentence,model): #function taking model and sentence from user
    p=create_bow(sentence,words) #calling the create bag of words with sentence and word
    res=model.predict(np.array([p]))[0] # making prediction with P and its 0th entry
    threshold=0.8 #defining the threshold 
    results=[[i,r] for i,r in enumerate(res) if r>threshold] #creating list with result of the model prediction
    results.sort(key=lambda x: x[1],reverse=True) # sorting the result
    
    return_list=[] #empty list
    for result in results: #for loop thru the result
        return_list.append({'intent':classes[result[0]],'prob':str(result[1])}) #append the intent classes result and result to the queary into the returm list
    return return_list #returing the list

In [64]:
#defing the response function
def get_response(return_list,intents_json):
  #giving the chatbot some basic tags for the extreme ondtion
    if len(return_list)==0: #no retunlist 
        tag='noanswer' #print(no answer)
    else:    
        tag=return_list[0]['intent'] #else return the retrun list intent value
        
        list_of_intents= intents_json['intents']    #making list of intents for json intents
    
        for i in list_of_intents: #looping thru the list of intents
            if tag==i['tag'] : #getting the tags
                result = random.choice(i['responses']) #making the random respose & assign to the result
    
        return result #retuning the result

In [65]:
#defining the chatbot function respose
def response(text):
    return_list=predict_class(text,model) #calling the predict call function for making the prediction for the given text
    response=get_response(return_list,intents) #calling the get response function for getting the intent and its response
    return response #return the response
print("Welcome Im Your Vitual Assitance, How May I Help you Today!\nHappy Learning!") #printing intro
while(1): #loop runs continues
    x=input("You:") # getting user inpout
    res = response(x) #call response
    print("\nBot:",res) #print the result as bot response
    if x.lower() in ['quit','bye','goodbye','get lost','see you']: # if x value is any of it in the list
        print("Thank you, Happy Learning!") #print comment
        break #break the loop

Welcome Im Your Vitual Assitance, How May I Help you Today!
Happy Learning!
You:help me understand NLP
1/1 [==============================] - 0s 86ms/step

Bot: None
You:i am not able to understand gradient boosting
1/1 [==============================] - 0s 38ms/step

Bot: Link: Machine Learning wiki 
You:ml
1/1 [==============================] - 0s 15ms/step

Bot: Link: Machine Learning wiki 
You:sl
1/1 [==============================] - 0s 17ms/step

Bot: Link: Machine Learning wiki 
You:i am not able to understand naive bayes
1/1 [==============================] - 0s 32ms/step

Bot: Link: Machine Learning wiki 
You:my problem is not solved
1/1 [==============================] - 0s 28ms/step

Bot: Tarnsferring the request to your PM
You:not a good solution
1/1 [==============================] - 0s 18ms/step

Bot: Tarnsferring the request to your PM
You:thanks a lot
1/1 [==============================] - 0s 17ms/step

Bot: I hope I was able to assist you, Good Bye
You:quit
1/1 [======